In [0]:
from datetime import date, timedelta

# Remove this before running Data Factory Pipeline
start_date = date.today() - timedelta(1)

silver_adls = "abfss://silver@deprojectearthquake.dfs.core.windows.net/"
gold_adls = "abfss://gold@deprojectearthquake.dfs.core.windows.net/"

silver_data = f"{silver_adls}earthquake_events_silver/"

In [0]:
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType
# Ensure the below library is installed on your cluster
import reverse_geocoder as rg
from datetime import date, timedelta

In [0]:
df = spark.read.parquet(silver_data).filter(col('time') > start_date)

In [0]:
df = df.limit(100) # added to speed up processings as during testing it was proving a bottleneck

In [0]:
def get_country_code(lat, lon):
    """
    Retrieve the country code for a given latitude and longitude.

    Parameters:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Returns:
    str: Country code of the location, retrieved using the reverse geocoding API.

    Example:
    >>> get_country_details(48.8588443, 2.2943506)
    'FR'
    """
    try:
        coordinates = (float(lat), float(lon))
        result = rg.search(coordinates)[0].get('cc')
        print(f"Processed coordinates: {coordinates} -> {result}")
        return result
    except Exception as e:
        print(f"Error processing coordinates: {lat}, {lon} -> {str(e)}")
        return None

In [0]:
# registering the udfs so they can be used on spark dataframes
get_country_code_udf = udf(get_country_code, StringType())

In [0]:
get_country_code(48.8588443, 2.2943506)

Processed coordinates: (48.8588443, 2.2943506) -> FR


'FR'

In [0]:
# adding country_code and city attributes
df_with_location = \
                df.\
                    withColumn("country_code", get_country_code_udf(col("latitude"), col("longitude")))

In [0]:
df_with_location.printSchema()

root
 |-- id: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- elevation: double (nullable = true)
 |-- title: string (nullable = true)
 |-- place_description: string (nullable = true)
 |-- sig: long (nullable = true)
 |-- mag: double (nullable = true)
 |-- magType: string (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- updated: timestamp (nullable = true)
 |-- country_code: string (nullable = true)



In [0]:
df_with_location.head()

Row(id='av93734213', longitude=-152.295, latitude=61.2895, elevation=-0.12, title='M 0.6 - 67 km WNW of Tyonek, Alaska', place_description='67 km WNW of Tyonek, Alaska', sig=6, mag=0.64, magType='ml', time=datetime.datetime(2025, 8, 25, 23, 47, 23, 860000), updated=datetime.datetime(2025, 8, 26, 4, 30, 44, 700000), country_code='US')

In [0]:
# adding significance classification
df_with_location_sig_class = \
                            df_with_location.\
                                withColumn('sig_class', 
                                            when(col("sig") < 100, "Low").\
                                            when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
                                            otherwise("High")
                                            )

In [0]:
df_with_location_sig_class.printSchema()

root
 |-- id: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- elevation: double (nullable = true)
 |-- title: string (nullable = true)
 |-- place_description: string (nullable = true)
 |-- sig: long (nullable = true)
 |-- mag: double (nullable = true)
 |-- magType: string (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- updated: timestamp (nullable = true)
 |-- country_code: string (nullable = true)
 |-- sig_class: string (nullable = false)



In [0]:
df_with_location_sig_class.head()

Row(id='av93734213', longitude=-152.295, latitude=61.2895, elevation=-0.12, title='M 0.6 - 67 km WNW of Tyonek, Alaska', place_description='67 km WNW of Tyonek, Alaska', sig=6, mag=0.64, magType='ml', time=datetime.datetime(2025, 8, 25, 23, 47, 23, 860000), updated=datetime.datetime(2025, 8, 26, 4, 30, 44, 700000), country_code='US', sig_class='Low')

In [0]:
# Save the transformed DataFrame to the Silver container
gold_output_path = f"{gold_adls}earthquake_events_gold/"

In [0]:
# Append DataFrame to Silver container in Parquet format
df_with_location_sig_class.write.mode('append').parquet(gold_output_path)